In [ ]:
# Imports
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
import os
import lz4.frame

In [2]:
# Encryption key (32 bytes for ChaCha20)
ENCRYPTION_KEY = b'\xc2\xfd\xf3v`I\xdb]a+M\xcf\x12;\xa9\x01A\x08\xdd\xdf\x89\x05\xdc\x05f\xa2,\xe1"I\x0ft'
print(len(ENCRYPTION_KEY))  # Should print 32

NONCE_SIZE = 12  # ChaCha20 nonce size

32


In [3]:
# Function to compress data using LZ4
def compress_data(data: bytes) -> bytes:
    return lz4.frame.compress(data)

# Function to decompress data using LZ4
def decompress_data(compressed_data: bytes) -> bytes:
    return lz4.frame.decompress(compressed_data)

In [4]:
# Function to encrypt data using ChaCha20-Poly1305
def encrypt_data(data: bytes) -> bytes:
    nonce = os.urandom(NONCE_SIZE)
    cipher = ChaCha20Poly1305(ENCRYPTION_KEY)
    encrypted_data = cipher.encrypt(nonce, data, None)  # None for associated data
    return nonce + encrypted_data


# Function to decrypt data using ChaCha20-Poly1305
def decrypt_data(encrypted_data: bytes) -> bytes:
    nonce = encrypted_data[:NONCE_SIZE]
    ciphertext = encrypted_data[NONCE_SIZE:]
    cipher = ChaCha20Poly1305(ENCRYPTION_KEY)
    return cipher.decrypt(nonce, ciphertext, None)  # None for associated data

In [6]:
try:
    # Encrypt data
    data = b"Hello Agent, this is a secret message!"
    compressed_data = compress_data(data)
    encrypted_data = encrypt_data(compressed_data)
    print(f"Encrypted data: {encrypted_data}")

    # Decrypt data
    decrypted_data = decrypt_data(encrypted_data)
    decompressed_data = decompress_data(decrypted_data)
    print(f"Decrypted data: {decompressed_data.decode()}")
except Exception as e:
    print(f"Error: {e}")

Encrypted data: b"\xbfMm\xf4\xf1\xd4\xa4U\xb9\x1f\x80#\xe9\xda\x01Q\xc1\xfcUG\xe9,\xb8\xab\xee\xd8yr\xe1?\xcc\xac\x03Wc\xfa\xc7w\xe2\x86\x19}\xe6\xf4\xcd\xd2\x15\xee\x8b\x1e\xd7)\xfd\xe8\x863*\x94[<\x16\x97\xaeU\xb9\xfaC\x041`_'\x8a\x9b#\xb4m$\xf0\xbe\xdb\xcd$\x13\xde\xe1\x97[\x16"
Decrypted data: Hello Agent, this is a secret message!
